<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" align="left" src="https://i.creativecommons.org/l/by-nc-sa/4.0/80x15.png" /></a>&nbsp;| [Emmanuel Rachelson](https://personnel.isae-supaero.fr/emmanuel-rachelson?lang=en) | <a href="https://supaerodatascience.github.io/machine-learning/">https://supaerodatascience.github.io/deep-learning/</a>

# Variations on neural networks and stochastic gradient descent

These exercises are meant to help you play around with the objects we have seen in class. They should help you get a more abstract and refined understanding of the mechanisms behind neural networks. Some are easy, others are harder and might require some thinking: this is intended to push you to consider different fascets of ANNs. Some answers are voluntarily very short to encourage discussion, either among students or with teaching assistants.

## From neural networks to computation graphs, and back

We have seen that a feed-forward neural network is a computation graph, organized in layers. Let $f(\cdot, \Theta)$ be a fully connected neural network parameterized by $\Theta$, made of $L$ layers, with $\theta_l$ the weights and biases of layer $l$. 
Let $\sigma_l$ be a Lipschitz continuous activation function. 
Then, the forward pass can be written:

\begin{align}
    x_0 &= [x \ 1]  \\
    z_l &= \theta_l \cdot x_{l-1} \\
    x_l &= \sigma_l\left(z_l\right) \\
    f(x, \Theta) &= x_L 
\end{align}

The notation $[x \ 1]$ for $x_0$ indicates that we append a constant "1" component to $x$. Similarly, when $\sigma_l$ includes a constant "1" component, the notation above permits seamless integration of the biases within $\theta_l$.

Let $d_l$ denote the number of neurons on layer $l$, including the "1" constant ($d_0-1$ is hence the number of input variables).

Note that $x$ is a matrix of dimension $(d_0-1)\times B$ where $B$ is the minibatch size.

<div class="alert alert-warning">
What is the dimension of $\theta_l$?
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>
$$\theta_l \in \mathbb{R}^{d_l \times d_{l-1}}$$
</details>

<div class="alert alert-warning">
Write the network's output $f(x;\Theta)$ as a composition of functions and matrix-vector products, using the $\sigma_l$ and $\theta_l$ notations.
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>
$$f(x;\Theta) = \sigma_L\left( \theta_L \cdot \sigma_{L-1} \left( \theta_{L-1} \cdot \sigma_{L-1} \left( \ldots \theta_0 \cdot x_0 \right) \right) \right)$$
</details>

Let $\ell(z,z')$ be the loss used to fit the neural network. 
Computation of the empirical risk $\mathcal{L}$ for a given minibatch is yet another computation graph.

<div class="alert alert-warning">
How is this graph related to that of $f(x, \Theta)$?
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>
Overall, this graph is a function of $\Theta$, $x$ and $y$.
The graph of $f(x, \Theta)$ is a subgraph of that of $\mathcal{L}(x,y,\Theta)$, since:
$$\mathcal{L}(x,y,\Theta) = \sum_{x,y} \ell\left( f(x, \Theta), y \right)$$
</details>

<div class="alert alert-warning">
Draw inspiration from the derivation in class of the backpropagation algorithm with a mean square error, to derive a generic formulation for backpropagation.
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>
The backward pass, recursively computes the $\nabla_{\theta_l} \mathcal{L} (\Psi(x, \Theta); y)$ gradients for each layer $l$, in order to update the network parameters as:

\begin{align}
    \delta_L &= \sigma_L'(z_L), \\
    \theta_{l} &\leftarrow \theta_l - \eta \left( \delta_l \cdot x_{l-1}^T \right) \nabla_{x_L} \mathcal{L},  \\
    \delta_{l-1} &= \sigma_{l-1}'(z_{l-1}) \circ \left( \delta_l \cdot \theta_{l-1}  \right), 
\end{align}
where $\circ$ is the Hadamard product. 
</details>

Overall, all that we have done in this example, is define a general computation graph $\mathcal{L}(x,y,\Theta)$, which is a composition of successive functions, and compute it's gradient $\nabla_\Theta \mathcal{L}(x,y,\Theta)$ by invoking the chain rule. It is important to note that the application of the chain rule is not specific to neural networks: it is applicable to any feedforward computational graph for which the $\sigma_l$ are differentiable. This more general context is called **[automatic differentiation](https://en.wikipedia.org/wiki/Automatic_differentiation)**. The backpropagation algorithm is a specific case of reverse mode automatic differentiation. If one considers computer programs which are composed of differentiable atomic building blocks, one defines the field of **[differentiable programming](https://en.wikipedia.org/wiki/Differentiable_programming)**. Modern neural network libraries are primarily automatic differentiation libraries, with an specific API for the application to neural networks.

## Computational cost of the forward and backward passes

We have seen that a feed-forward neural network is a computation graph, organized in layers. Let $f(\cdot, \Theta)$ be a fully connected neural network parameterized by $\Theta$, made of $L$ layers, with $\theta_l$ the weights and biases of layer $l$. 
Let $\sigma_l$ be a Lipschitz continuous activation function. 
Then, the forward pass can be written:

\begin{align}
    x_0 &= [x \ 1]  \\
    z_l &= \theta_l \cdot x_{l-1}\\
    x_l &= \sigma_l\left(z_l\right)\\
    f(x, \Theta) &= x_L 
\end{align}

The notation $[x \ 1]$ for $x_0$ indicates that we append a constant "1" component to $x$. Similarly, when $\sigma_l$ includes a constant "1" component, the notation above permits seamless integration of the biases within $\theta_l$.

For the sake of simplicity, let $n$ be the number of neurons on each hidden layer, let $B$ be the mini-batch size and let $x_L$ have dimension one.

Let $\mathcal{L}$ be the empirical risk used to fit the neural network. 
The backward pass, recursively computes the $\nabla_{\theta_l} \mathcal{L} (\Psi(x, \Theta); y)$ gradients for each layer $l$, in order to update the network parameters as:

\begin{align}
    \delta_L &= \sigma_L'(z_L), \\
    \theta_{l} &\leftarrow \theta_l - \eta \left( \delta_l \cdot x_{l-1}^T \right) \nabla_{x_L} \mathcal{L},  \\
    \delta_{l-1} &= \sigma_{l-1}'(z_{l-1}) \circ \left( \delta_l \cdot \theta_{l-1}  \right), 
\end{align}
where $\circ$ is the Hadamard product. 

<div class="alert alert-warning">
What is the time complexity of the forward pass?
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>
$z_l = \theta_l \cdot x_{l-1}$ induces $O(n^2B)$ operations.  
$x_l = \sigma_l\left(z_l\right)$, in turn, induces $O(nB)$ operations.  
Overall, the forward pass induces $O(Ln^2B)$ operations.
</details>

<div class="alert alert-warning">
What is the time complexity of the backward pass?
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>
$\theta_{l} \leftarrow \theta_l - \eta \left( \delta_l \cdot x_{l-1}^T \right) \nabla_{x_L} \mathcal{L}$ induces $O(n^2B)$ operations.  
$\delta_{l-1} = \sigma_{l-1}'(z_{l-1}) \circ \left( \delta_l \cdot \theta_{l-1}  \right)$ induces $O((n + n^2)B)$ operations.  
Hence the backward pass induces $O(2Ln^2B)$ operations.
</details>

The point of this simple reminder is to recall that the complexity classes of both operations are overall the same, but that the backward pass requires (an order of) twice as many operations as the forward pass. 

Of course, parallelization and caching in modern computation architectures may greatly amortize this complexity and make it unnoticeable in some cases.

<div class="alert alert-warning">
Verify this experimentally using the code developped in class.
</div>

## Convergence speed of SGD

Consider a stochastic gradient descent process, minimizing a regression loss for function $f_\theta$, currently at step $k$. 
Let $\theta^*$ denote the optimal solution to the optimization problem.
The current parameter is $\theta_k$.
A mini-batch has been drawn from the training set and yielded a gradient estimate $g_k$. 

Given a gradient step $\eta$, one has $\theta_{k+1} = \theta_k - \eta g_k$.

Let us define the convergence speed $S_k$ of stochastic gradient descent as:
$$S_k = -\mathbb{E}_{\textrm{minibatch} \sim \bar{p}} \left[ \|\theta_{k+1} - \theta^*\|_2^2 - \|\theta_k - \theta^*\|_2^2 \right].$$

Here, the expectation is taken with respect to the minibatch drawn, $\bar{p}$ being the empirical distribution of the training set. 
This quantity measures how much closer to $\theta^*$ we can expect $\theta_{k+1}$ to be, compared to $\theta_k$, on average when drawing mini-batches.

Let us drop the $k$ index and write 
$$S = -\mathbb{E}_{\textrm{minibatch} \sim \bar{p}} \left[ \|\theta - \eta g - \theta^*\|_2^2 - \|\theta - \theta^*\|_2^2 \right].$$

<div class="alert alert-warning">
Write $S$ as an expression depending of $g$. There are two parts in this expression, what does each represent? In particular, how is one of the two terms connected to the distribution of losses values across the training set, at the current $\theta$?
</div>

<details class="alert alert-danger">
    <summary markdown="span"><b>Ready to see the answer? (click to expand)</b></summary>
\begin{align}
S &= -\mathbb{E} \left[ \|\theta - \eta g - \theta^*\|_2^2 - \|\theta - \theta^*\|_2^2 \right]\\
&= -\mathbb{E} \left[ \eta^2 g^T g - 2(\theta-\theta^*)^T g \right]\\
&= 2 (\theta-\theta^*)^T \mathbb{E} \left[ g \right] -\mathbb{E} \left[ g^T g \right]
\end{align}

The first term is $\theta$-dependent and relies on the expected gradient across minibatches. This expected gradient is always the same, whatever the batch size or the batch sampling method. So this term measures really how much we can gain on average for the considered function in the specific current $\theta$.

The second term however is the expected value, across mini-batches, of the square of the gradient norm $\mathbb{E} \left[ \| g \|^2_2\right]$.
If $\theta$ was one-dimensional, then so would be the gradient. And in this case, $\mathbb{E} \left[g^2\right]$ would be $Var(g) + \mathbb{E}[g]^2$. Specifically, this second term would indicate the variance of the gradient norm. If the gradient norm had large variance across mini-batches, the convergence speed would be bad.

Now $\theta$ is multi-dimensional and the relationship is a bit more complex with $\mathbb{E} \left[ \| g \|^2_2\right] = \textrm{Tr}( \mathbb{V} [g]) + \mathbb{E}[g]^T \mathbb{E} [g]$, with $\mathbb{V} [g]$ the covariance matrix of the elements of $g$. 
Despite this somehow more convoluted expression, the interpretation is the same: this second term indicate a notion of variance of the gradient norm. 
Again, if the gradient norm has large variance across mini-batches, the convergence speed is smaller.

When does that happen? This happens for instance when training errors are quite unbalanced across the training set. Suppose that for the current $\theta$, we fit perfectly every single $(x,y)$ in the training set, but one which we write $\tilde{x},\tilde{y}$ for which we make an arbitrarily large error. Specifically, write $\tilde{f} = f_\theta(\tilde{x}$, then we suppose $\nabla_\tilde{f} \ell(\tilde{f},\tilde{y})$ is arbitrarily large. Then, if this sample is not picked in the minibatch, the gradient is exactly zero. But if $\tilde{x},\tilde{y}$ is picked, then the gradient is non-zero and its norm is proportional to $\nabla_\tilde{f} \ell(\tilde{f},\tilde{y})$. These discrepancies induce a large variability in the gradient's norm, which penalizes the convergence speed.
</details>

This short exercise was inpired by these two papers:  
Wang, L., Yang, Y., Min, R., & Chakradhar, S. (2017). [Accelerating deep neural network training with inconsistent stochastic gradient descent](https://arxiv.org/abs/1603.05544). Neural Networks, 93, 219-229.   
Katharopoulos, A., & Fleuret, F. (2018). [Not all samples are created equal: Deep learning with importance sampling](https://arxiv.org/abs/1803.00942). In International conference on machine learning.

In particular, the second paper notes that the second term we found above can be increased by sampling mini-batches non uniformly in the training set. 

<div class="alert alert-warning">
Read the 3 first pages of the paper (until the end of Section 3.2) and implement their method using the code developped in class (discard Section 3.3 to keep things relatively simple).
</div>